In [1]:
from datasets import load_dataset
from evaluate import load
from pathlib import Path
import os
import yaml
import re

In [3]:
path_to_predictions = ''
path_to_save_metrics = ''

In [4]:
predictions_dataset = load_dataset('json', data_files=path_to_predictions)['train']

In [5]:
def split(example):
    pattern = r'[a-z]+[0-9]*|[A-Z](?:[a-z]+[0-9]*|[A-Z]*[0-9]*(?=[A-Z]|$))'
    example['words_labels'] = ' '.join(re.findall(pattern, example['labels'])).lower()
    example['words_predictions'] = ' '.join(re.findall(pattern, example['prediction'])).lower()
    return example

In [6]:
predictions_dataset = predictions_dataset.map(split)

In [102]:
rouge = load('rouge')

In [103]:
rouge_results = rouge.compute(
    predictions=predictions_dataset['words_predictions'],
    references=predictions_dataset['words_labels']
)

for k, v in rouge_results.items():
    rouge_results[k] = float(v)

In [104]:
Path(path_to_save_metrics).mkdir(parents=True, exist_ok=True)

In [105]:
metrics = {
    "rouge_results": rouge_results
}

In [107]:
with open(os.path.join(path_to_save_metrics, 'metrics.yml'), 'w') as f:
    yaml.dump(metrics, f)